# Running Tune experiments with Skopt

This example demonstrates the usage of Skopt with Ray Tune via `SkOptSearch`.

It also also shows that a scheduler can be used with `SkOptSearch`, e.g. `AsyncHyperBandScheduler`.

Click below to see all the imports we need for this example.
You can also launch directly into a Binder instance to run this notebook yourself.
Just click on the rocket symbol at the top of the navigation.

Necessary requirements:
- `pip install ray[tune]`
- `pip install scikit-optimize==0.8.1`

In [ ]:
# !pip install ray[tune]
!pip install scikit-optimize==0.8.1

In [ ]:
import time
from typing import Dict, Optional, Any

import ray
from ray import tune
from ray.tune.suggest import ConcurrencyLimiter
from ray.tune.schedulers import AsyncHyperBandScheduler
from ray.tune.suggest.skopt import SkOptSearch

In [ ]:
ray.init(configure_logging=False)

Let's start by defining a simple evaluation function.
An explicit math formula is queried here for demonstration, yet in practice this is typically a black-box function-- e.g. the performance results after training an ML model.
We artificially sleep for a bit (`0.1` seconds) to simulate a long-running ML experiment.
This setup assumes that we're running multiple `step`s of an experiment while tuning three hyperparameters,
namely `width`, `height`, and `activation`.

In [ ]:
def evaluate(step, width, height, activation):
    time.sleep(0.1)
    activation_boost = 10 if activation=="relu" else 0
    return (0.1 + width * step / 100) ** (-1) + height * 0.1 + activation_boost

Next, our ``objective`` function to be optimized takes a Tune ``config``, evaluates the `score` of your experiment in a training loop,
and uses `tune.report` to report the `score` back to Tune.

In [ ]:
def objective(config):
    for step in range(config["steps"]):
        score = evaluate(step, config["width"], config["height"], config["activation"])
        tune.report(iterations=step, mean_loss=score)
              

Next we define a search space. The critical assumption is that the optimal hyperparamters live within this space. Yet, if the space is very large, then those hyperparamters may be difficult to find in a short amount of time.

#The simplest case is a search space with independent dimensions. In this case, a config dictionary will suffice.

In [ ]:
search_space = {
    "steps": 100,
    "width": tune.uniform(0, 20),
    "height": tune.uniform(-100, 100),
    "activation": tune.choice(["relu", "tanh"]),
}

Here we define the Skopt search algorithm:

In [ ]:
algo = SkOptSearch(
    # parameter_names=space.keys(),  # If you want to set the space
    # parameter_ranges=space.values(), # If you want to set the space
    # points_to_evaluate=previously_run_params,
    # evaluated_rewards=known_rewards
)

We also constrain the the number of concurrent trials to `4` with a `ConcurrencyLimiter`.

In [ ]:
algo = ConcurrencyLimiter(algo, max_concurrent=4)


Lastly, we set the number of samples for this Tune run to `1000`
(you can decrease this if it takes too long on your machine).

In [ ]:
num_samples = 1000

In [ ]:
# We override here for our smoke tests.
num_samples = 10

Furthermore, we define a `scheduler` to go along with our algorithm. This is optional, and only to demonstrate that we don't need to compromise other great features of Ray Tune while using Skopt.

In [ ]:
scheduler = AsyncHyperBandScheduler()

Now all that's left is running the experiment.

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    scheduler=scheduler,
    metric="mean_loss",
    mode="min",
    name="skopt_exp",
    num_samples=num_samples,
    config=search_space
)

We now have hyperparameters found to minimize the mean loss.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

## Providing an initial set of hyperparameters

While defining the search algorithm, we may choose to provide an initial set of hyperparameters that we believe are especially promising or informative, and
pass this information as a helpful starting point for the `SkOptSearch` object. We also can pass the known rewards for these initial params to save on unnecessary computation.

In [ ]:
initial_params = [
    {"width": 10, "height": 0, "activation": "relu"},
    {"width": 15, "height": -20, "activation": "tanh"}
]
known_rewards = [-189, -1144]

Now the `search_alg` built using `SkOptSearch` takes `points_to_evaluate`.

In [ ]:
algo = SkOptSearch(points_to_evaluate=initial_params)
algo = ConcurrencyLimiter(algo, max_concurrent=4)

And run the experiment with initial hyperparameter evaluations:

In [ ]:
analysis = tune.run(
    objective,
    search_alg=algo,
    metric="mean_loss",
    mode="min",
    name="skopt_exp_with_warmstart",
    num_samples=num_samples,
    config=search_space
)

And we again show the ideal hyperparameters.

In [ ]:
print("Best hyperparameters found were: ", analysis.best_config)

In [ ]:
ray.shutdown()